### Imports & Settings

In [1]:
# Load data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Dummy baseline
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### Load Data

In [9]:
# Get url from file hosted on GitHub
url = "https://raw.githubusercontent.com/ZocoMacc/car-quirks-ml_InnovaLab25/refs/heads/main/data/cars_data.csv"

# Cargar csv en un DataFrame
df = pd.read_csv(url)

# Explorar la estructura del DataFrame (Sanity Check)
df.info()
df.shape
# df.isna().sum().sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   name                    10000 non-null  object 
 1   year                    10000 non-null  int64  
 2   selling_price           10000 non-null  int64  
 3   km_driven               10000 non-null  int64  
 4   fuel                    10000 non-null  int64  
 5   combustible_estimado_l  10000 non-null  float64
 6   seller_type             10000 non-null  int64  
 7   transmission            10000 non-null  int64  
 8   owner                   10000 non-null  int64  
 9   tipo_carroceria         10000 non-null  int64  
 10  potencia_motor_hp       10000 non-null  int64  
 11  nivel_seguridad         10000 non-null  float64
 12  calidad_auto            10000 non-null  object 
 13  score_calidad           10000 non-null  float64
 14  eficiencia_km_l         10000 non-null 

(10000, 15)

In [10]:
# Ver el balance de las clases (84% Media, 10.5% Alta, 5.5% Baja)
df["calidad_auto"].value_counts(normalize=True)

,proportion
calidad_auto,
Media,0.8399
Alta,0.1052
Baja,0.0549


#### Lo que esto nos dice
- Un fuerte desbalanceo de clases
  - "Media" es la mayoria con un 84%
  - "Alta" solo cubre el 10%, y "Baja" solo un 5.5%
- El accuracy de referencia (baseline) es engañosamente alto
  - Un DummyClassifier que siempre predice "Media" deberia de poder predecir 83.99%.
  - Superar este baseline no garantiza que el modelo sea mas precizo detectando "Alta" o "Baja".
- Riesgo de abandono de las clases minoritarias
  - El modelo tiene el riesgo de predecir "Media" la mayoria del tiempo debido al desbalanceo de las clases, lo que podria ignorar "Alta" o "Baja" como opciones.

### Cleaning & Dropping
- Decidir las variables que seran ignoradas por el modelo.
- Si es necesarion, transformar valores en interpretaciones legibles por el modelo o marcarlas con una categoria.

In [11]:
# Duda sobre la variable "score_calidad": Que tanto se correlaciona con "calidad_auto"
# Mapear las categorias con numeros para calcular la correlacion
mapping = {"Baja": 0, "Media": 1, "Alta": 2}
df["calidad_code"] = df["calidad_auto"].map(mapping)

# Calcular la correlacion
corr = df[["score_calidad", "calidad_code"]].corr().iloc[0, 1]

# Imprimir los resultados
print(f"Correlacion entre score_calidad y calidad_auto: {corr:.3f}")

Correlacion entre score_calidad y calidad_auto: 0.740


##### Lo que esto nos dice
- La correlacion entre "score_calidad" y "calidad_auto" no es 1.0, lo que quiere decir que "calidad_auto" no depende al 100% de la variable "score_calidad".
- La variable "score_calidad" va a ser considerada.

#### Extraer marca del nombre
- Usar el nombre y cortarlo de una manera en la que podamos extrar la marca del auto para poder usarlo como variable y ser considerada.

In [12]:
"""
Rifense este rollo porfi
"""

'\nRifense este rollo porfi\n'

#### Convertir columnas de enteros en categorias
- Columnas como "fuel", "seller_type", "transmission", "owner", "tipo_carroceria" tienen asignado un valor entero pero en realidad representan categorias sin orden.
- Convertir estas variables a Pandas' category dtype para mas claridad y eficiencia en la manipulacion de estas variables, tambien es mas seguro hacerlo de esta forma, asi evitamos que estas columnas se interpreten como numeric imputers o scalers.

In [13]:
"""
Ahorita me lo aviento -macc
"""

'\nAhorita me lo aviento -macc\n'

### DummyClassifier & Baseline
#### Establecer una Dummy Baseline
- El primer paso es establecer una baseline y asegurarse de que la informacion esta limpia
- El objetivo es confirmar ques este DummyClassifier de como resultado una precision de 83.99%
- Se usara la estrategia de "most_frequent", simplemente el valor que mas se repite sera elegido para la prediccion.
- Este modelo representa el piso (baseline) de qualquier modelo futuro.



In [14]:
# Establecer la Dummy Baseline
X = df.drop("calidad_auto", axis=1)
y = df["calidad_auto"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,       # preserva los ratios 84/10.5/5.5 para train y test
    random_state=42
)

# Iniciar DummyClassifier
dummy = DummyClassifier(
    strategy="most_frequent", # Siempre predice la clase mas frecuente ("Media")
    random_state=42
)
dummy.fit(X_train, y_train)   # "Entrena" basado en la clase mas frecuente

# Obtener predicciones
y_pred = dummy.predict(X_test)  # Aplica la regla a cada fila de X_test

# Evaluar accuracy
accuracy = accuracy_score(y_test, y_pred)   # Fraccion de predicciones correctas
print(f"DummyClassifier Accuracy: {accuracy}")

# Visualizar resulatdos de predicciones
print("Classification report:\n")
print(classification_report(
    y_test,
    y_pred,
    digits=4,         # Numero de decimales
    zero_division=0   # Ignorar zero-division errors (debido a strings)
))

# Generar confusion matrix
cm = confusion_matrix(y_test, y_pred, labels=["Media","Alta","Baja"])
print("Confusion Matrix:\n", cm)

DummyClassifier Accuracy: 0.84
Classification report:

              precision    recall  f1-score   support

        Alta     0.0000    0.0000    0.0000       210
        Baja     0.0000    0.0000    0.0000       110
       Media     0.8400    1.0000    0.9130      1680

    accuracy                         0.8400      2000
   macro avg     0.2800    0.3333    0.3043      2000
weighted avg     0.7056    0.8400    0.7670      2000

Confusion Matrix:
 [[1680    0    0]
 [ 210    0    0]
 [ 110    0    0]]


#### Lo que esto nos dice
- Tenemos un baseline de ~84% accuracy, este es el objetivo a vencer en los proximos modelos.

### Features

### Modeling

### Evaluation